<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/emergent_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# emergent_ai.py

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import random
import os

# 1. Synthetic Dataset for Continuous Self-Emergence
class EmergentDataset(Dataset):
    """
    Generates synthetic data (X, Y) where:
      Y1 = sin(x0 * x1)                   # self-emergence flux
      Y2 = cos(x2 + x3)                   # stability metric
      Y3 = Y1 * Y2                        # dissolution equilibrium
    """
    def __init__(self, num_samples=2000, input_dim=6, seed=42):
        super().__init__()
        random.seed(seed)
        np.random.seed(seed)
        self.X = np.random.uniform(-2, 2, (num_samples, input_dim)).astype(np.float32)
        y1 = np.sin(self.X[:,0] * self.X[:,1])
        y2 = np.cos(self.X[:,2] + self.X[:,3])
        y3 = y1 * y2
        self.Y = np.stack([y1, y2, y3], axis=1).astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.from_numpy(self.Y[idx])

# 2. Model Definition with Dropout for Uncertainty
class EmergentAI(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, p_dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(p_dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.drop(x)
        return self.fc2(x)

# 3. Physics-informed Residual: Enforce Y1 * Y2 == Y3
def physics_residual(y_pred):
    return y_pred[:,0] * y_pred[:,1] - y_pred[:,2]

# 4. Training & Evaluation Utilities
def train_one_epoch(model, loader, optimizer, loss_fn, lambda_phy, device):
    model.train()
    running_loss = 0.0
    for Xb, Yb in loader:
        Xb, Yb = Xb.to(device), Yb.to(device)
        optimizer.zero_grad()
        preds = model(Xb)
        mse = loss_fn(preds, Yb)
        phy = torch.mean(physics_residual(preds)**2)
        loss = mse + lambda_phy * phy
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * Xb.size(0)
    return running_loss / len(loader.dataset)

def evaluate(model, loader, loss_fn, lambda_phy, device):
    model.eval()
    tot_loss = tot_mse = tot_phy = 0.0
    with torch.no_grad():
        for Xb, Yb in loader:
            Xb, Yb = Xb.to(device), Yb.to(device)
            preds = model(Xb)
            mse = loss_fn(preds, Yb)
            phy = torch.mean(physics_residual(preds)**2)
            tot_loss += (mse + lambda_phy * phy).item() * Xb.size(0)
            tot_mse  += mse.item() * Xb.size(0)
            tot_phy  += phy.item() * Xb.size(0)
    n = len(loader.dataset)
    return tot_loss/n, tot_mse/n, tot_phy/n

def mc_dropout_predict(model, X, T=50):
    model.train()  # keep dropout active
    sims = []
    for _ in range(T):
        sims.append(model(X).unsqueeze(0))
    sims = torch.cat(sims, dim=0)  # [T, batch, out_dim]
    return sims.mean(dim=0), sims.std(dim=0)

# 5. Main Training Loop
def main():
    # Hyperparameters
    input_dim, hidden_dim, output_dim = 6, 64, 3
    batch_size, epochs = 64, 200
    lr, lambda_phy = 1e-3, 1.0
    patience = 20
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Prepare dataset & loaders
    dataset = EmergentDataset(num_samples=2000, input_dim=input_dim)
    train_len = int(0.8 * len(dataset))
    val_len   = len(dataset) - train_len
    train_ds, val_ds = torch.utils.data.random_split(dataset, [train_len, val_len])
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_ds, batch_size=batch_size)

    # Model, loss, optimizer, scheduler
    model     = EmergentAI(input_dim, hidden_dim, output_dim, p_dropout=0.1).to(device)
    loss_fn   = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)

    # Training with early stopping
    best_val = float('inf')
    wait     = 0
    history  = {'train_loss': [], 'val_loss': []}

    for epoch in range(1, epochs+1):
        tr_loss = train_one_epoch(model, train_loader, optimizer,
                                  loss_fn, lambda_phy, device)
        val_loss, _, _ = evaluate(model, val_loader, loss_fn,
                                  lambda_phy, device)
        scheduler.step(val_loss)

        history['train_loss'].append(tr_loss)
        history['val_loss'].append(val_loss)
        print(f"Epoch {epoch:03d} | Train {tr_loss:.4f} | Val {val_loss:.4f}")

        if val_loss < best_val:
            best_val = val_loss
            torch.save(model.state_dict(), "best_emergent_ai.pth")
            wait = 0
        else:
            wait += 1
            if wait > patience:
                print("Early stopping triggered.")
                break

    # Load best model
    model.load_state_dict(torch.load("best_emergent_ai.pth"))

    # 6. Visualization
    # Loss curves
    plt.figure()
    plt.plot(history['train_loss'], label='Train')
    plt.plot(history['val_loss'],   label='Val')
    plt.yscale('log')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training & Validation Loss')
    plt.savefig('loss_curve_emergent.png', dpi=150)

    # Parity plot
    model.eval()
    Xv, Yv = next(iter(val_loader))
    Xv, Yv = Xv.to(device), Yv.to(device)
    preds = model(Xv)
    plt.figure(figsize=(6,6))
    plt.scatter(Yv.cpu().numpy().ravel(), preds.detach().cpu().numpy().ravel(), alpha=0.5)
    mn, mx = float(min(Yv.min(), preds.min())), float(max(Yv.max(), preds.max()))
    plt.plot([mn, mx], [mn, mx], 'r--')
    plt.xlabel('True')
    plt.ylabel('Predicted')
    plt.title('Parity Plot — EmergentAI')
    plt.savefig('parity_emergent.png', dpi=150)

    # Uncertainty histogram for output Y1
    mean_pred, std_pred = mc_dropout_predict(model, Xv, T=50)
    u = std_pred[:,0].detach().cpu().numpy()
    plt.figure()
    plt.hist(u, bins=30, color='teal')
    plt.xlabel('Std Dev of Y1')
    plt.title('Uncertainty Distribution (Y1)')
    plt.savefig('uncertainty_emergent.png', dpi=150)

    print("Done. Saved model and plots:\n"
          "  best_emergent_ai.pth\n"
          "  loss_curve_emergent.png\n"
          "  parity_emergent.png\n"
          "  uncertainty_emergent.png")

if __name__ == '__main__':
    main()